In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

In [2]:
def binary_map_create(img):
    """
    Creat a binary map. The pixel have the value [201,242,208] will be mapped as -1
    :param img:
    :return:
    """
    rows, cols, _ = img.shape
    binary_img = np.zeros((rows,cols))
    for row in range(rows):
        for col in range(cols):
            if (img[row,col] == [201,242,208]).all():
                binary_img[row,col] = -1
    return binary_img

In [3]:
def neighbor_value(img, label_d, binary_img: np.array, offsets, reverse=False):
    """
    Find the connected pixel where all have the same value and label them
    """
    rows, cols = binary_img.shape
    label_idx = 0
    rows_ = [0, rows, 3] if reverse == False else [rows-3, 0, -3]
    cols_ = [0, cols, 3] if reverse == False else [cols-3, 0, -3]
    for row in range(rows_[0], rows_[1], rows_[2]):
        for col in range(cols_[0], cols_[1], cols_[2]):
            label = 0
            if (img[row,col] != [201,242,208]).any():
                continue
            for offset in offsets:
                neighbor_row = min(max(0, row + offset[0]), rows - 1)
                neighbor_col = min(max(0, col + offset[1]), cols - 1)
                neighbor_val = img[neighbor_row, neighbor_col]
                if (neighbor_val != [201,242,208]).any():
                    continue
                neighbor_label = binary_img[neighbor_row,neighbor_col]
                if neighbor_label == -1 and label == 0:
                    label = -1
                elif neighbor_label > 0 and label == 0:
                    label = neighbor_label
                elif neighbor_label == -1 and label > 0:
                    continue
                elif neighbor_label > 0 and label > 0:
                    label = min(neighbor_label,label)
            if label == -1:
                label_idx += 1
                label = label_idx
            if label not in label_d:
                label_d[label] = [(row,col)]
            elif (row,col) not in label_d[label]:
                label_d[label].append((row,col))
            if binary_img[row,col] != label and int(binary_img[row,col]) in label_d:
                del_index = label_d[int(binary_img[row,col])].index((row,col))
                label_d[int(binary_img[row,col])].pop(del_index)
            binary_img[row][col] = label
        if row%300 == 0:
            print("%d done" %(row))
    return binary_img, label_d

In [4]:
def label_d_fix(d):
    """
    Delete the area that contains less pixel
    """
    del_list = []
    for ele in d:
        if len(d[ele]) <20:
            del_list.append(ele)
    for ele in del_list:
        del d[ele]
    return d

In [5]:
def Two_Pass(img: np.array):
    """
    Find and label connected area
    """
    offsets = [[0, -3], [-3, 0], [0, 0], [3, 0], [0, 3]]
    binary_img = binary_map_create(img)
    label_d = {}
    binary_img,label_d = neighbor_value(img, label_d, binary_img, offsets, False)
    binary_img,label_d = neighbor_value(img, label_d, binary_img, offsets, True)
    label_d = label_d_fix(label_d)

    return binary_img, label_d

In [6]:
for name in ['bronx','brooklyn','manhattan','queens','staten_island']:
    filename = './taxi_zone_map_'+name+'.jpg'
    image = io.imread(filename)
    binary_img, label_d = Two_Pass(image)
    savename_d = './'+name+'_label_d'
    savename_b = './'+name+'_binary'
    np.save(savename_d,label_d)
    np.save(savename_b,binary_img)

0 done
300 done
600 done
900 done
1200 done
1500 done
1800 done
2100 done
2400 done
2700 done
3000 done
3000 done
2700 done
2400 done
2100 done
1800 done
1500 done
1200 done
900 done
600 done
300 done
0 done
300 done
600 done
900 done
1200 done
1500 done
1800 done
2100 done
2400 done
2700 done
3000 done
3000 done
2700 done
2400 done
2100 done
1800 done
1500 done
1200 done
900 done
600 done
300 done
0 done
300 done
600 done
900 done
1200 done
1500 done
1800 done
2100 done
2400 done
2700 done
3000 done
3000 done
2700 done
2400 done
2100 done
1800 done
1500 done
1200 done
900 done
600 done
300 done
0 done
300 done
600 done
900 done
1200 done
1500 done
1800 done
2100 done
2400 done
2700 done
3000 done
3000 done
2700 done
2400 done
2100 done
1800 done
1500 done
1200 done
900 done
600 done
300 done


In [ ]:
['brooklyn','manhattan','queens','staten_island']
label_d=np.load('./bronx_label_d')
for label in label_d.keys():
    for loc in label_d[lebel]:
        img[loc] = [155,0,0]

In [12]:
# Label map to the zone area. The key is the zone ID. Value is the tuple for city and their related labels
label_zone_d={200:('bronx',2),240:('bronx',25),259:('bronx',86),220:('bronx',303),
              241:('bronx',428),174:('bronx',513),254:('bronx',284),81:('bronx',379),51:('bronx',(595,632)),
             184:('bronx',(661,995,1698)),46:('bronx',(2078,2081,2434,2635)),3:('bronx',992),32:('bronx',1055),31:('bronx',(938,962,998)),
             20:('bronx',1243),18:('bronx',(728,1030)),136:('bronx',868),94:('bronx',(1352,1507)),185:('bronx',1736),
             183:('bronx',(2034,2292)),58:('bronx',2530),208:('bronx',2738),213:('bronx',(3516,3532)),250:('bronx',2677),
             182:('bronx',2695),242:('bronx',(1998,2015,2049)),212:('bronx',3142),248:('bronx',(2747,2922)),
             60:('bronx',(2660,2671)),78:('bronx',1841),47:('bronx',(1979,2046,2123,2246,2483)),59:('bronx',(2383,2581)),126:('bronx',3368),
             147:('bronx',(3254,3638)),199:('bronx',4135),168:('bronx',3625),159:('bronx',3389),167:('bronx',(2874,3032)),
             69:('bronx',(2283,2369,2472,2588,2606,2861)),247:('bronx',(2147,2571,2580,2642,2684,3055)),119:('bronx',(2083,2202,)),169:('bronx',1681),235:('bronx',1301),
              112:('brooklyn',1),255:('brooklyn',(73,104)),80:('brooklyn',(76,77,80,93,98,143,182,221)),256:('brooklyn',(187,200)),
              217:('brooklyn',309),37:('brooklyn',262),36:('brooklyn',390),66:('brooklyn',(308,310)),
              34:('brooklyn',(365,387)),33:('brooklyn',354),65:('brooklyn',460),97:('brooklyn',597),
              49:('brooklyn',651),17:('brooklyn',505),225:('brooklyn',682),63:('brooklyn',1044),
              177:('brooklyn',(1189,1507)),61:('brooklyn',1263),189:('brooklyn',(1138,1406)),25:('brooklyn',736),
              52:('brooklyn',(741,936)),54:('brooklyn',(701,903)),40:('brooklyn',982),195:('brooklyn',(868,1004)),
              106:('brooklyn',(1167,1173,1355)),181:('brooklyn',(1067,1102,1445)),190:('brooklyn',(1556,1803)),62:('brooklyn',1665),
              188:('brooklyn',2002),35:('brooklyn',1772),77:('brooklyn',1732),76:('brooklyn',1562),222:('brooklyn',2774),
              72:('brooklyn',2098),71:('brooklyn',2444),85:('brooklyn',(2443,2617)),89:('brooklyn',2395),
              257:('brooklyn',(2010,2373)),111:('brooklyn',2055),228:('brooklyn',(1593,1627,1840,1935,2197)),227:('brooklyn',(2338,2498,2607)),
              133:('brooklyn',2671),26:('brooklyn',2779),39:('brooklyn',2687),91:('brooklyn',3156),
              155:('brooklyn',(3563,3831)),165:('brooklyn',3483),178:('brooklyn',(3552,3561)),22:('brooklyn',(3501,3530)),
              11:('brooklyn',3813),67:('brooklyn',(3178,3473)),14:('brooklyn',(2560,2719)),21:('brooklyn',(3856,4038,4249)),
              123:('brooklyn',4003),149:('brooklyn',3976),108:('brooklyn',4449),210:('brooklyn',(4421,4442)),
              29:('brooklyn',4653),55:('brooklyn',4764),150:('brooklyn',4969),154:('brooklyn',(4204,4493)),
              153:('manhattan',47),127:('manhattan',(89,138)),128:('manhattan',(3,34,56)),243:('manhattan',120),
              120:('manhattan',(215,353)),244:('manhattan',(300,356)),116:('manhattan',438),152:('manhattan',(557,569)),
              42:('manhattan',(452,464,512,537,603)),166:('manhattan',(623,720)),41:('manhattan',741),74:('manhattan',797),
              194:('manhattan',(973,1020,1229)),24:('manhattan',(806,876)),151:('manhattan',848),
              43:('manhattan',(933,939,978,1015,1037,1090,1122,1266,1306,1338,1384,1404)),
              75:('manhattan',1019),238:('manhattan',938),239:('manhattan',1102),143:('manhattan',(1344,1468)),
              142:('manhattan',(1409,1601)),236:('manhattan',(1251,1449)),237:('manhattan',(1597,1826)),263:('manhattan',(1352,1557,1561)),
              141:('manhattan',(1685,1954)),262:('manhattan',(1439,1635)),140:('manhattan',(1768,1962)),202:('manhattan',(1800,2233,2249)),
              50:('manhattan',(1613,1781)),246:('manhattan',(1965,2298)),48:('manhattan',(1754,1974)),68:('manhattan',(2186,2528)),
              163:('manhattan',(1834,1957)),230:('manhattan',(1950,2116)),100:('manhattan',(2216,2343)),186:('manhattan',(2433,2555)),
              90:('manhattan',(2633,2744)),161:('manhattan',(2013,2187)),164:('manhattan',(2335,2620)),234:('manhattan',(2693,2845)),
              162:('manhattan',(2079,2336)),170:('manhattan',(2424,2626)),107:('manhattan',(2789,2929)),229:('manhattan',(2140,2337)),
              233:('manhattan',(2392,2561)),137:('manhattan',(2738,2861)),224:('manhattan',(3004,3140)),158:('manhattan',(2740,2963)),
              249:('manhattan',(2808,2980)),113:('manhattan',(2921,2948)),114:('manhattan',(3040,3174)),79:('manhattan',(3037,3056)),
              4:('manhattan',3227),125:('manhattan',(3125,3253)),211:('manhattan',(3184,3320)),144:('manhattan',(3305,3402)),
              148:('manhattan',(3363,3480)),232:('manhattan',(3453,3529)),231:('manhattan',3240),45:('manhattan',3509),
              209:('manhattan',3630),13:('manhattan',(3428,3623)),261:('manhattan',(3584,3689)),87:('manhattan',(3659,3725,3738)),
              12:('manhattan',0),88:('manhattan',3736),105:('manhattan',(3826,3838)),103:('manhattan',0),95:('queens',1933),
              104:('manhattan',3788),145:('queens',878),193:('queens',(477,713)),146:('queens',(776,853,888)),179:('queens',(176,336)),
              8:('queens',(115,187)),223:('queens',6),7:('queens',459),226:('queens',(1020,1031)),260:('queens',(768,1388,1546)),
              207:('queens',(622,634,794)),138:('queens',(209,259)),129:('queens',558),70:('queens',538),83:('queens',(1343,1646)),
              82:('queens',(1390,1793,1841)),157:('queens',(1875,2283,2450)),160:('queens',2199),198:('queens',(2521,2632)),
              96:('queens',(2984,2992,3117,3127,3155,3164)),102:('queens',(2819,2887,2943,3001,3018)),
              196:('queens',(2039,2147)),56:('queens',(1482)),173:('queens',1067),57:('queens',0),
              93:('queens',991),253:('queens',(884,1091)),92:('queens',(579,625)),53:('queens',13),252:('queens',1),
              15:('queens',(104,228,328)),16:('queens',649),171:('queens',436),73:('queens',1151),9:('queens',899),
              192:('queens',1483),64:('queens',584),101:('queens',(1576,1593)),19:('queens',1946),175:('queens',(1371,2123)),
              98:('queens',1981),131:('queens',2526),121:('queens',2097),135:('queens',1824),134:('queens',2831),
              28:('queens',2795),191:('queens',2602),122:('queens',(2912,2976)),130:('queens',(3015,3026,3069,3099,3177)),197:('queens',3191),
              258:('queens',(3288,3411)),180:('queens',3745),124:('queens',3865),216:('queens',3612),10:('queens',(3715,3726)),
              215:('queens',(3345,3355,3374)),205:('queens',3231),38:('queens',3496),139:('queens',(3816,3852)),30:('queens',(4905,5133,5166)),
              218:('queens',3905),219:('queens',4138),203:('queens',(3991,4006,4048,4304,4394)),132:('queens',(4100,4152)),
              2:('queens',(4509,4519,4529,4608,4610,4614,4621,4732,4814,4823,4844,4882,5026,5049,5068,5236)),
              86:('queens',5009),117:('queens',(5064,5181,5208,5335)),201:('queens',(5389,5421,5428,5433,5452)),
              156:('staten_island',39),27:('queens',(5484,5488,5500,5523,5527,5530,5540)),
              187:('staten_island',(118,516,676)),251:('staten_island',(410,447)),245:('staten_island',(174,219)),
              206:('staten_island',(1,25,125,134)),115:('staten_island',(338,356,368,391,397,445,668)),221:('staten_island',(239,691)),
              23:('staten_island',322),99:('staten_island',(1164,1540,1579,1587,1602,1618,1634,1648,1678)),
              6:('staten_island',1054),214:('staten_island',(1195,1202,1212)),172:('staten_island',1582),
              176:('staten_island',(1731,1777,1779)),110:('staten_island',(2168,2246,2586,2590)),109:('staten_island',1844),
              5:('staten_island',(1935,1947,1953,1975,1982,2013)),118:('staten_island',(967,1192,1269,1276,1281,1359)),
              84:('staten_island',(2056,2080,2119,2126,2159,2245,2454,2472,2494,2524,2535,2542,2549,2560,2569,2625,2634,2645,2708,2709,2720,2722,2742,2743,2751)),
              204:('staten_island',2146),44:('staten_island',(2180,2270,2284,2367,2389,2404))}